In [1]:
import numpy as np
import psycopg2
import pickle
import logging
import dask.dataframe as dd
from sqlalchemy import create_engine
from dask.distributed import Client

In [2]:
def get_db_connection():
    conn = psycopg2.connect(
        dbname='simDB',
        user='user',
        password='password',
        host='localhost',
        port='5432'
    )
    return conn

def fetch_trial_data(conn, trial_id):
    try:
        cur = conn.cursor()
        
        # Fetch data for the given trial_id
        cur.execute('SELECT timestep, simulation_time, input_data, output_data, state_data FROM simulation_data WHERE trial_id = %s', (trial_id,))
        rows = cur.fetchall()
        
        # Deserialize data
        data = []
        for row in rows:
            timestep, simulation_time, input_data_bin, output_data_bin, state_data_bin = row
            input_data = pickle.loads(input_data_bin)
            output_data = pickle.loads(output_data_bin)
            state_data = pickle.loads(state_data_bin)
            data.append((timestep, simulation_time, input_data, output_data, state_data))
        
        cur.close()
        return data
    except Exception as e:
        print(f"Error fetching data for trial_id {trial_id}: {e}")
        return None


In [3]:

# Example usage
conn = get_db_connection()

trial_id = 5  # Replace with the trial_id you want to fetch
trial_data = fetch_trial_data(conn, trial_id)

if trial_data:
    for timestep, simulation_time, input_data, output_data, state_data in trial_data:
        print(f"Timestep: {timestep}, Simulation Time: {simulation_time}")
        print(f"Input Data: {input_data}")
        print(f"Output Data: {output_data}")
        print(f"State Data: {state_data}")

conn.close()


Timestep: 0, Simulation Time: 0.0
Input Data: [ 0.         -0.          0.00173205 -0.00173205 -0.         -0.        ]
Output Data: [-1.11451102e+03 -4.99902312e-02 -1.05807786e+03 -4.70731034e-02
 -9.99437047e+02  1.62218132e-02 -9.40638447e+02  1.91366911e-01
 -8.81635061e+02  1.76194116e-01 -8.23557966e+02 -3.34750121e-01
 -7.65174327e+02 -2.19597813e-01 -7.04452050e+02 -1.68051014e-01
 -6.46401608e+02 -1.97499434e+00 -5.88391892e+02 -5.36474242e-01
 -5.29513012e+02  5.24098327e-01 -4.71151916e+02 -2.22499587e+00
 -4.10736484e+02 -9.67685358e-02 -3.53540939e+02  1.56018823e-01
 -2.94493450e+02 -1.79999742e+00 -2.35372931e+02 -3.19011578e-01
 -1.76998405e+02 -3.04685831e-01 -1.19423732e+02  2.15671647e-01
 -5.87456325e+01 -7.56153459e-02 -4.11639879e+00  1.88168741e-01]
State Data: [[-1.48000000e+02 -5.21609373e-05  4.99955557e+01]
 [-1.48000000e+02 -5.21609368e-05 -4.99955531e+01]
 [-1.48000000e+02  2.69359719e+00  4.99665128e+01]
 ...
 [-2.09165016e+02  4.50430985e+01 -3.36437269e

In [4]:
# Connect to the Dask cluster (optional, for distributed computing)
client = Client()

In [5]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 6
Total threads: 24,Total memory: 125.48 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:35157,Workers: 6
Dashboard: http://127.0.0.1:8787/status,Total threads: 24
Started: Just now,Total memory: 125.48 GiB
Comm: tcp://127.0.0.1:43841,Total threads: 4
Dashboard: http://127.0.0.1:41581/status,Memory: 20.91 GiB
Nanny: tcp://127.0.0.1:43561,


In [6]:
# Database connection parameters
db_user = 'user'
db_password = 'password'
db_host = 'localhost'
db_port = '5432'
db_name = 'simDB'

# Create the connection engine
engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')


In [8]:

# Define the table name
table_name = 'simulation_data'
connection_string = f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}'

# Read data into a Dask DataFrame
# Adjust the `npartitions` and `bytes_per_chunk` as needed for your dataset
ddf = dd.read_sql_table(table_name, connection_string, npartitions=100, index_col='id', bytes_per_chunk=5e6)

# Example operation: Compute the mean of a column
mean_values = ddf['simulation_time'].mean().compute()
print(mean_values)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 0: invalid start byte